In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

dataset = pd.read_csv("nba2.csv")

def highest_paid_player():
    highest_paid = dataset.loc[dataset["Salary"].idxmax(), ["Name", "Team", "Salary"]]
    print("Highest-Paid Player:")
    print(highest_paid)
    main_menu()

def filters_funtion():
    filter_input = '1'
    filter_list = []
    
    while filter_input != '0':
        print("\n--- Available Filters ---")
        print("1. Name")
        print("2. Team")
        print("3. Number")
        print("4. Position")
        print("5. Age")
        print("6. Weight")
        print("0. Done - Apply filters")
        print()
        filter_input = input("Enter the number of filter to apply: ")
        print()
        
        if filter_input != '0':
            filter_list.append(filter_input)
    
    result = dataset.copy()

    for filters in filter_list:
        if filters == '1':
            name_filter = input("Enter name (partial match): ").capitalize()
            result = result.loc[result["Name"].str.contains(name_filter, case=False, na=False)]
            
            if result.empty:
                print(f"No players found matching name: {name_filter}")
                break
                
        elif filters == '2':
            team_filter = input("Enter team name: ").upper()
            result = result.loc[result["Team"].str.contains(team_filter, case=False, na=False)]
            
            if result.empty:
                print(f"No players found for team: {team_filter}")
                break
                
        elif filters == '3':
            number_filter = input("Enter jersey number: ")
            try:
                result = result.loc[result["Number"] == float(number_filter)]
                
                if result.empty:
                    print(f"No players found with number: {number_filter}")
                    break
            except ValueError:
                print("Invalid number entered. Skipping this filter.")
                
        elif filters == '4':
            position_filter = input("Enter position (e.g., PG, SG, SF, PF, C): ").upper()
            result = result.loc[result["Position"].str.contains(position_filter, case=False, na=False)]
            
            if result.empty:
                print(f"No players found for position: {position_filter}")
                break
                
        elif filters == '5':
            print("Age filter options:")
            print("1. Exact age")
            print("2. Age range")
            age_option = input("Choose option: ")
            
            if age_option == '1':
                age_filter = input("Enter age: ")
                try:
                    result = result.loc[result["Age"] == float(age_filter)]
                    
                    if result.empty:
                        print(f"No players found with age: {age_filter}")
                        break
                except ValueError:
                    print("Invalid age entered. Skipping this filter.")
                    
            elif age_option == '2':
                try:
                    min_age = float(input("Enter minimum age: "))
                    max_age = float(input("Enter maximum age: "))
                    result = result.loc[(result["Age"] >= min_age) & (result["Age"] <= max_age)]
                    
                    if result.empty:
                        print(f"No players found in age range: {min_age}-{max_age}")
                        break
                except ValueError:
                    print("Invalid age range entered. Skipping this filter.")

        else:
            print("Invalid filter choice. Skipping.")
    
    if not result.empty:
        print("\n" + "="*80)
        print(f"FILTERED RESULTS ({len(result)} player(s) found):")
        print("="*80)
        print(result.to_string(index=False))
        print("-"*80 + "\n")
    else:
        print("\nNo players match all the applied filters.")
    
    main_menu()

def compare_players():
    print("\n" + "="*80)
    print("PLAYER COMPARISON")
    print("="*80)
    
    num_players = input("How many players do you want to compare (2-5)? ")
    
    try:
        num_players = int(num_players)
        if num_players < 2 or num_players > 5:
            print("Please enter a number between 2 and 5.")
            return
    except ValueError:
        print("Invalid input. Please enter a number.")
        return
    
    players_to_compare = []
    
    for i in range(num_players):
        print(f"\n--- Player {i+1} ---")
        player_name = input(f"Enter name of player {i+1}: ").strip()
        
        matches = dataset.loc[dataset["Name"].str.contains(player_name, case=False, na=False)]
        
        if matches.empty:
            print(f"No player found matching '{player_name}'. Skipping.")
            continue
        elif len(matches) > 1:
            print(f"\nMultiple players found matching '{player_name}':")
            print(matches[["Name", "Team"]].to_string(index=True))
            index_choice = input("Enter the index number of the player you want: ")
            try:
                selected_player = matches.loc[int(index_choice)]
                players_to_compare.append(selected_player)
            except (ValueError, KeyError):
                print("Invalid index. Skipping this player.")
        else:
            players_to_compare.append(matches.iloc[0])
            print(f"✓ Added: {matches.iloc[0]['Name']}")
    
    if len(players_to_compare) < 2:
        print("\nNeed at least 2 valid players to compare. Please try again.")
        return
    
    comparison_df = pd.DataFrame(players_to_compare)
    
    print("\n" + "="*80)
    print("PLAYER COMPARISON RESULTS")
    print("="*80 + "\n")
    
    comparison_transposed = comparison_df.T
    comparison_transposed.columns = [f"Player {i+1}" for i in range(len(players_to_compare))]
    
    print(comparison_transposed.to_string())
    
    print("\n" + "-"*80)
    print("QUICK ANALYSIS")
    print("-"*80)
    
    if "Salary" in comparison_df.columns:
        salaries = comparison_df["Salary"].dropna()
        if not salaries.empty:
            highest_paid_idx = salaries.idxmax()
            print(f"💰 Highest Paid: {comparison_df.loc[highest_paid_idx, 'Name']} - ${salaries.max():,.2f}")
            
            lowest_paid_idx = salaries.idxmin()
            print(f"💵 Lowest Paid: {comparison_df.loc[lowest_paid_idx, 'Name']} - ${salaries.min():,.2f}")
            
            salary_diff = salaries.max() - salaries.min()
            print(f"📊 Salary Difference: ${salary_diff:,.2f}")
    
    if "Age" in comparison_df.columns:
        ages = comparison_df["Age"].dropna()
        if not ages.empty:
            oldest_idx = ages.idxmax()
            youngest_idx = ages.idxmin()
            print(f"\n👴 Oldest: {comparison_df.loc[oldest_idx, 'Name']} - {int(ages.max())} years")
            print(f"👶 Youngest: {comparison_df.loc[youngest_idx, 'Name']} - {int(ages.min())} years")
    
    print("\n" + "="*80 + "\n")
    
    # Visualization
    viz_choice = input("Would you like to see visual comparison? (Y/N): ").upper()
    if viz_choice == 'Y':
        visualize_comparison(comparison_df)
    
    main_menu()

def visualize_comparison(comparison_df):
    """Create visual comparison of players"""
    players = comparison_df['Name'].values
    num_players = len(players)
    
    # Create figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Player Comparison Dashboard', fontsize=16, fontweight='bold')
    
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8'][:num_players]
    
    # Plot 1: Salary Comparison
    ax1 = axes[0, 0]
    if 'Salary' in comparison_df.columns:
        salaries = comparison_df['Salary'].values
        bars1 = ax1.bar(range(num_players), salaries, color=colors, edgecolor='black', linewidth=1.5)
        ax1.set_xticks(range(num_players))
        ax1.set_xticklabels(players, rotation=45, ha='right')
        ax1.set_ylabel('Salary ($)', fontsize=11, fontweight='bold')
        ax1.set_title('💰 Salary Comparison', fontsize=12, fontweight='bold')
        ax1.grid(axis='y', alpha=0.3, linestyle='--')
        
        # Add value labels on bars
        for i, bar in enumerate(bars1):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height,
                    f'${height:,.0f}', ha='center', va='bottom', fontsize=9)
    
    # Plot 2: Age Comparison
    ax2 = axes[0, 1]
    if 'Age' in comparison_df.columns:
        ages = comparison_df['Age'].values
        bars2 = ax2.bar(range(num_players), ages, color=colors, edgecolor='black', linewidth=1.5)
        ax2.set_xticks(range(num_players))
        ax2.set_xticklabels(players, rotation=45, ha='right')
        ax2.set_ylabel('Age (years)', fontsize=11, fontweight='bold')
        ax2.set_title('👤 Age Comparison', fontsize=12, fontweight='bold')
        ax2.grid(axis='y', alpha=0.3, linestyle='--')
        
        # Add value labels on bars
        for i, bar in enumerate(bars2):
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height)}', ha='center', va='bottom', fontsize=10)
    
    # Plot 3: Weight Comparison
    ax3 = axes[1, 0]
    if 'Weight' in comparison_df.columns:
        weights = comparison_df['Weight'].values
        bars3 = ax3.bar(range(num_players), weights, color=colors, edgecolor='black', linewidth=1.5)
        ax3.set_xticks(range(num_players))
        ax3.set_xticklabels(players, rotation=45, ha='right')
        ax3.set_ylabel('Weight (lbs)', fontsize=11, fontweight='bold')
        ax3.set_title('⚖️ Weight Comparison', fontsize=12, fontweight='bold')
        ax3.grid(axis='y', alpha=0.3, linestyle='--')
        
        # Add value labels on bars
        for i, bar in enumerate(bars3):
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height)}', ha='center', va='bottom', fontsize=10)
    
    # Plot 4: Player Info Summary
    ax4 = axes[1, 1]
    ax4.axis('off')
    
    summary_text = "📋 PLAYER DETAILS\n" + "─"*40 + "\n\n"
    for i, player in enumerate(players):
        summary_text += f"● {player}\n"
        summary_text += f"  Team: {comparison_df.iloc[i]['Team']}\n"
        summary_text += f"  Position: {comparison_df.iloc[i]['Position']}\n"
        summary_text += f"  Number: #{int(comparison_df.iloc[i]['Number']) if pd.notna(comparison_df.iloc[i]['Number']) else 'N/A'}\n"
        summary_text += "\n"
    
    ax4.text(0.05, 0.95, summary_text, transform=ax4.transAxes,
            fontsize=10, verticalalignment='top', family='monospace',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()

def fitness_profile_generator():
    print("\n" + "="*80)
    print("FITNESS PROFILE GENERATOR")
    print("="*80)
    
    player_name = input("\nEnter player name: ").strip()
    
    matches = dataset.loc[dataset["Name"].str.contains(player_name, case=False, na=False)]
    
    if matches.empty:
        print(f"No player found matching '{player_name}'.")
        return
    elif len(matches) > 1:
        print(f"\nMultiple players found matching '{player_name}':")
        print(matches[["Name", "Team", "Position", "Age"]].to_string(index=True))
        index_choice = input("\nEnter the index number of the player you want: ")
        try:
            player = matches.loc[int(index_choice)]
        except (ValueError, KeyError):
            print("Invalid index.")
            return
    else:
        player = matches.iloc[0]
    
    name = player["Name"]
    age = player["Age"]
    position = player["Position"]
    team = player["Team"]
    number = player["Number"] if pd.notna(player["Number"]) else "N/A"
    salary = player["Salary"] if pd.notna(player["Salary"]) else 0
    height_str = player["Height"] if pd.notna(player["Height"]) else None
    weight = player["Weight"] if pd.notna(player["Weight"]) else None
    
    height_inches = None
    height_display = "N/A"
    if height_str:
        try:
            parts = str(height_str).split("-")
            if len(parts) == 2:
                feet = int(parts[0])
                inches = int(parts[1])
                height_inches = (feet * 12) + inches
                height_display = f"{feet}'{inches}\" ({height_inches} inches)"
        except:
            height_display = str(height_str)
    
    bmi = None
    bmi_category = None
    if height_inches and weight:
        try:
            weight_kg = float(weight) * 0.453592
            height_m = height_inches * 0.0254
            bmi = weight_kg / (height_m ** 2)
            
            if bmi < 18.5:
                bmi_category = "🔵 UNDERWEIGHT - May lack muscle mass"
            elif 18.5 <= bmi < 25:
                bmi_category = "🟢 NORMAL - Healthy range for athletes"
            elif 25 <= bmi < 27:
                bmi_category = "🟡 ATHLETIC BUILD - High muscle mass typical for NBA"
            elif 27 <= bmi < 30:
                bmi_category = "🟠 MUSCULAR/HEAVY - Common for forwards/centers"
            else:
                bmi_category = "🔴 VERY HEAVY - High mass, monitor for joint stress"
        except:
            pass
    
    print("\n" + "="*80)
    print(f"FITNESS PROFILE: {name}")
    print("="*80)
    print(f"Team: {team}")
    print(f"Position: {position}")
    print(f"Jersey Number: {number}")
    print(f"Age: {int(age)} years")
    print(f"Salary: ${salary:,.2f}")
    
    print("\n" + "-"*80)
    print("PHYSICAL MEASUREMENTS")
    print("-"*80)
    print(f"Height: {height_display}")
    print(f"Weight: {weight} lbs" if weight else "Weight: N/A")
    
    if bmi:
        print(f"BMI: {bmi:.2f}")
        print(f"BMI Category: {bmi_category}")
    
    print("\n" + "="*80 + "\n")
    main_menu()

def main_menu():
    main_menu_input = ""
    while main_menu_input not in ['Y', 'N']:
        print()
        main_menu_input = input("RETURN TO MAIN MENU (Y/N): ").capitalize()
        if main_menu_input == 'Y':
            main()
        elif main_menu_input == 'N':
            print("Bye Bye 👋")
            print()

def main():
    print("1. Apply Filters")
    print("2. Compare Players")
    print("3. Fitness Profile")
    print("4. Highest paid player")
    print()
    userInput = input("Enter choice: ")

    if userInput == '1':
        filters_funtion()
    elif userInput == '2':
        compare_players()
    elif userInput == '3':
        fitness_profile_generator()
    elif userInput == '4':
        highest_paid_player()
    else:
        print("Invalid choice!")

main()

1. Apply Filters
2. Compare Players
3. Fitness Profile
4. Highest paid player


FITNESS PROFILE GENERATOR

FITNESS PROFILE: Kobe Bryant
Team: Los Angeles Lakers
Position: SF
Jersey Number: 24.0
Age: 37 years
Salary: $25,000,000.00

--------------------------------------------------------------------------------
PHYSICAL MEASUREMENTS
--------------------------------------------------------------------------------
Height: 6'6" (78 inches)
Weight: 212.0 lbs
BMI: 24.50
BMI Category: 🟢 NORMAL - Healthy range for athletes
Ideal BMI for Position: 24-27
Assessment: ✓ Ideal BMI for small forward

--------------------------------------------------------------------------------
BODY COMPOSITION ANALYSIS
--------------------------------------------------------------------------------
Build Type: 💪 BALANCED - Good strength-to-speed ratio
Weight-to-Height Ratio: 2.72 lbs/inch
Expected Weight Range for Position: 206-226 lbs
Weight Status: ✓ OPTIMAL WEIGHT - Well-suited for position

-----------------